In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import vibrationtesting as vt
from vibrationtesting import sos_modal
import matplotlib.pyplot as plt
import scipy
import numpy as np
import scipy.io as sio
import array_to_latex as a2t
np.set_printoptions(precision = 9, linewidth = 220, suppress = True)

Convert complex modes to real modes

In [4]:
M = np.array([[4, 0, 0],
              [0, 4, 0],
              [0, 0, 4]])
Cso = np.array([[1,0,0],
                [0,0,0],
                [0,0,0]])
K = np.array([[8, -4, 0],
              [-4, 8, -4],
              [0, -4, 4]])
omega, _, Psi_true = vt.sos_modal(M, K)
Bt = np.array([[1],[0],[0]])
Ca = np.array([[1,0,0]])
Cd = Cv = np.zeros_like(Ca)
A, B, C, D = vt.so2ss(M, Cso, K, Bt, Cd, Cv, Ca)
Am, Bm, Cm, Dm, eigenvalues, modes = vt.ss_modal(A, B, C, D)
mat_contents=sio.loadmat('1stNaturalFreq.mat')
U1 = mat_contents['U1']
complex_modes = modes[:3,0::2]
complex_modes = complex_modes@np.linalg.inv(np.diag(np.diag(complex_modes)))
Psi_est = vt.real_modes(U1)
Psi=Psi_est

In [5]:
mat_contents=sio.loadmat('WingBeamModal.mat')
K = mat_contents['K']
M = mat_contents['M']
#omega, zeta, Psi = vt.sos_modal(scipy.sparse.linalg([M]), scipy.sparse.linalg([K]))
#scipy.sparse.linalg(K)
K

<141x141 sparse matrix of type '<class 'numpy.float64'>'
	with 1627 stored elements in Compressed Sparse Column format>

In [7]:
 measured = np.array([[3,9,15,21,27,33,39,45,51,57,63,69,75,81,87,93,99,105]])
 omega=13.54
 #Psi_measured = np.array([U1])
 #Psi_full = vt.mode_expansion_from_model(Psi_measured, omega[0], M, K, measured)
 #Psi_measured
 vt.mode_expansion_from_model(Psi, omega, M, K, measured)

NotImplementedError: Reshaping not implemented for csr_matrix.

In [13]:
complex_modes

array([[ 1.000000000-0.j         ,  2.099234839-0.684531423j,  1.678352494+0.52790572j ],
       [ 1.798247408+0.099411477j,  1.000000000-0.j         , -2.218125195-0.153628421j],
       [ 2.240934713+0.157537198j, -1.651470818+0.512991869j,  1.000000000-0.j         ]])

In [5]:
Psi_true

array([[-0.163992639,  0.368488115, -0.295504524],
       [-0.295504524,  0.163992639,  0.368488115],
       [-0.368488115, -0.295504524, -0.163992639]])

In [6]:
Psi_real = np.real(complex_modes)
Psi_real

array([[ 1.         ,  2.099234839,  1.678352494],
       [ 1.798247408,  1.         , -2.218125195],
       [ 2.240934713, -1.651470818,  1.         ]])

In [77]:
a2t.to_ltx(Psi_true, frmt = '{:3.5f}')

\begin{bmatrix}
 -0.16399,  0.36849, -0.29550\\
 -0.29550,  0.16399,  0.36849\\
 -0.36849, -0.29550, -0.16399
\end{bmatrix}


In [8]:
help(a2t.to_clp)

Help on function to_clp in module array_to_latex.array_to_latex:

to_clp(a, frmt='{:1.2f}', arraytype='bmatrix')
    Returns a LaTeX array the the clipboard given a numpy array
    
    Parameters
    ----------
    a         : array
    frmt      : python 3 formatter, optional
                https://mkaz.tech/python-string-format.html
    arraytype : latex array type- `bmatrix` default, optional
    
    Returns:
    --------
    out: str
        LaTeX array
    
    See Also
    --------
    array_to_latex
    
    Examples
    ________
    >>> import numpy as np
    >>> import array_to_latex as ar
    >>> A = np.array([[1.23456, 23.45678],[456.23, 8.239521]])
    >>> ar.to_clp(A, frmt = '{:6.2f}', arraytype = 'array')
    
    Note that the output is in your clipboard, so you won't see any results.
    See `to_ltx` for further examples



In [9]:
Psi_abs = np.abs(complex_modes)*np.real(np.sign(complex_modes))
Psi_abs

array([[ 1.         ,  2.208024044,  1.759417956],
       [ 1.800993165,  1.         , -2.223439019],
       [ 2.246465302, -1.729310996,  1.         ]])

In [10]:
Psi_est

array([[ 1.         ,  2.208024044,  1.759417956],
       [ 1.80253425 ,  0.974920375, -2.116079409],
       [ 2.229245067, -1.760956913,  0.902996771]])

In [11]:
vt.mac(Psi_true,Psi_real)

array([[0.999999338, 0.000479191, 0.000063106],
       [0.000000603, 0.999343589, 0.000762406],
       [0.000000059, 0.00017722 , 0.999174488]])

In [73]:
a2t.to_ltx(vt.mac(Psi_true,Psi_real), frmt = '{:3.5f}')

\begin{bmatrix}
  1.00000,  0.00048,  0.00006\\
  0.00000,  0.99934,  0.00076\\
  0.00000,  0.00018,  0.99917
\end{bmatrix}


In [67]:
vt.mac(Psi_true,Psi_abs)

array([[0.99999997 , 0.000187273, 0.         ],
       [0.         , 0.999812655, 0.000061695],
       [0.00000003 , 0.000000072, 0.999938305]])

In [71]:
a2t.to_ltx(vt.mac(Psi_true,Psi_abs), frmt = '{:3.5f}')

\begin{bmatrix}
  1.00000,  0.00019,  0.00000\\
  0.00000,  0.99981,  0.00006\\
  0.00000,  0.00000,  0.99994
\end{bmatrix}


In [15]:
vt.mac(Psi_true,Psi_est)

array([[0.999983388, 0.000000759, 0.000007767],
       [0.000012366, 0.999990366, 0.000566347],
       [0.000004246, 0.000008875, 0.999425887]])

In [72]:
a2t.to_ltx(vt.mac(Psi_true,Psi_est), frmt = '{:3.5f}')

\begin{bmatrix}
  0.99998,  0.00000,  0.00001\\
  0.00001,  0.99999,  0.00057\\
  0.00000,  0.00001,  0.99943
\end{bmatrix}


In [17]:
Psi_real = vt.mass_normalize(Psi_real, M)

In [18]:
Psi_real.T@M@Psi_real

array([[ 1.         ,  0.022663407, -0.007723197],
       [ 0.022663407,  1.         , -0.041083449],
       [-0.007723197, -0.041083449,  1.         ]])

In [76]:
a2t.to_ltx(Psi_real.T@M@Psi_real)

\begin{bmatrix}
  1.00,  0.02, -0.01\\
  0.02,  1.00, -0.04\\
 -0.01, -0.04,  1.00
\end{bmatrix}


In [75]:
a2t.to_ltx(complex_modes, frmt = '{:3.5f}')

\begin{bmatrix}
  0.16435+0.00000j,  0.36802-0.68453j,  0.28391+0.52791j\\
  0.29554+0.09941j,  0.17531-0.00000j, -0.37521-0.15363j\\
  0.36830+0.15754j, -0.28952+0.51299j,  0.16916-0.00000j
\end{bmatrix}
